In [146]:
import numpy as np
from numpy.random import binomial
from scipy.stats import gamma, beta, expon, poisson, uniform, bernoulli
from scipy.special import gamma as gammaFunc
import random
import pandas as pd

In [267]:
import numpy as np
from numpy.random import binomial

def simulate_epidemic(N=1000000, T_max=100):
    Delta = np.tile(np.array([1] + [0] * 24), 4)
    Stage = np.cumsum(Delta)
    K = sum(Delta)

    b = 1 / np.array([0.3, 0.4, 0.25, 0.2])
    r = 1 / (1 - np.array([0.05, 0.15, 0.2, 0.25])) - 1

    beta = 1 / b[Stage-1]
    gamma = r[Stage-1] / (1 + r[Stage-1])

    I_current = 50
    R_current = 0
    S_current = N - I_current - R_current

    N_infect_obs = [I_current]
    N_recovery_obs = [R_current]

    for t in range(0, T_max):
        PI_current = I_current / N
        p_beta = 1 - np.exp(-beta[t] * PI_current)
        New_infect = binomial(S_current, p_beta)
        New_recovery = binomial(I_current, gamma[t])

        N_infect_obs.append(New_infect)
        N_recovery_obs.append(New_recovery)
        S_current -= New_infect
        I_current += New_infect - New_recovery
        R_current += New_recovery

    S_obs = N - np.cumsum(N_infect_obs)
    I_obs = np.cumsum( np.array(N_infect_obs)-  np.array(N_recovery_obs))
    R_obs = np.cumsum(N_recovery_obs)
    PI_obs = np.array(I_obs)/N

    N_infect_obs = N_infect_obs
    N_recovery_obs = N_recovery_obs
    
    df = pd.DataFrame({
        'susceptible': S_obs,
        'infects': I_obs,
        'recovered': R_obs,
        'PI': PI_obs,
        'deltaI': N_infect_obs,
        'deltaR': N_recovery_obs
    })
    return df

# Example usage:
result = simulate_epidemic()

In [268]:
result

,susceptible,infects,recovered,PI,deltaI,deltaR
0,999950,50,0,0.000050,50,0
1,999935,62,3,0.000062,15,3
2,999909,82,9,0.000082,26,6
3,999896,89,15,0.000089,13,6
4,999874,103,23,0.000103,22,8
...,...,...,...,...,...,...
96,176742,52,823206,0.000052,3,15
97,176741,41,823218,0.000041,1,12
98,176739,35,823226,0.000035,2,8
99,176737,30,823233,0.000030,2,7


In [263]:
def gibbs_sampling(data,samples = 10000,T_max=100,burnin=1000):
    
    ############# data
    # Data is expected to be a dataframe with 101 rows (100 steps + initial one)
    # OBS: the initial row should display the starting values 
    #      for Susceptible (S), Infected (I), and Recovered (R), with zero values in the 
    #      columns representing changes (deltas) over time.
    
    I_obs = data["infects"].values
    S_obs = data["susceptible"].values
    PI_obs = data["PI"].values
    N_infect_obs = data["deltaI"].values[1:]
    N_recovery_obs = data["deltaR"].values[1:]
    
    ############# Hyperparameters
    p_a = 1/T_max**4
    p_b = 2 - p_a
    
    b_shape = 0.1
    b_rate = 0.1
    r_shape = 0.1
    r_rate = 0.1
    
    gamma_b_shape = gammaFunc(b_shape)
    gamma_r_shape = gammaFunc(r_shape)
    
    ############# initialization
    
    #delta
    Delta_hat = np.array([1] + [0] * (T_max - 2) + [0]).flatten()
    Stage_hat = np.cumsum(Delta_hat)
    K_hat     = np.sum(Delta_hat)

    #b and r
    b_hat    = gamma.rvs(a=b_rate, scale=b_shape, size=K_hat)
    r_hat    = gamma.rvs(a=r_rate, scale=r_shape, size=K_hat)
    
    #beta and gamma
    beta_hat    = np.zeros(T_max)
    gamma_hat   = np.zeros(T_max)
    contact_hat = np.zeros(T_max)

    for t in range(T_max):
        beta_hat[t]  = expon.rvs(scale=1/b_hat[Stage_hat[t]-1])
        gamma_hat[t] = beta.rvs(a=r_hat[Stage_hat[t]-1], b=1)

        lambda_t       = beta_hat[t] * PI_obs[t]
        p_upper        = 1 - poisson.cdf(0, lambda_t)
        pp             = uniform.rvs(size= N_infect_obs[t]) * p_upper + (1 - p_upper)
        contact_hat[t] = np.sum(poisson.ppf(pp, lambda_t))

    for t in range(T_max):

        beta_hat[t]  = gamma.rvs(contact_hat[t] + 1, scale=1 / (b_hat[Stage_hat[t]-1] + PI_obs[t] * S_obs[t]))
        gamma_hat[t] = beta.rvs(N_recovery_obs[t] + r_hat[Stage_hat[t]-1], 1 + I_obs[t] - N_recovery_obs[t])

        lambda_t              = beta_hat[t] * PI_obs[t]
        p_upper               = 1 - poisson.cdf(0, lambda_t)
        pp                    = uniform.rvs(size = N_infect_obs[t]) * p_upper + (1 - p_upper)
        contact_hat[t]        = np.sum(poisson.ppf(pp, lambda_t))

    print("Initialization:\n")
    print("Delta_hat:",Delta_hat)
    print("b_hat:",b_hat)
    print("r_hat:",r_hat)
    print("beta_hat:",beta_hat)
    print("gamma_hat:",gamma_hat)
    print("----------------------------------------------------")
    print("----------------------------------------------------")
    
    ############# sampling 
    
    Delta_all = []
    Stage_all = []
    b_all     = []
    r_all     = []
    beta_all  = []
    gamma_all = []
    
    for step in range(samples):
        
        ############# delta_hat sampling
        ## add–delete–swap - proposal step
        # -1 delete
        # 0 swap
        # +1 add
        
        change_type = random.choice([-1, 0, 1])
        
        if K_hat==1:
            change_type = 1
        
        if K_hat==T_max:
            change_type = -1
            
        print("Step:",step)
        print("Change_type:",change_type)
        print("K_hat:",K_hat)
        
        if change_type == 1:
            
            Delta_hat_candidate = Delta_hat.copy()
            Stage_hat_candidate = Stage_hat.copy()
        
            zero_indices = np.where(Delta_hat[1:] == 0)[0]+1
            index_to_change = np.random.choice(zero_indices)

            Delta_hat_candidate[index_to_change] = 1
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)
            
            index_change_candidate = np.where(Delta_hat_candidate == 1)[0]
            change_position_all_candidate = np.where(Delta_hat_candidate[1:] == 1)[0] + 1
    
            phase_original = np.array([Stage_hat[index_to_change]])
            phase_candidate = np.array([Stage_hat[index_to_change], Stage_hat_candidate[index_to_change]])
            
            logp_original = 0
            logp_candidate = 0
            
            
            for i in phase_original:
                L_i_original = np.where(Stage_hat == i)[0]
                
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(1, len(L_i_original))))
                
            for i in phase_candidate:
                L_i_candidate = np.where(Stage_hat_candidate == i)[0]
                
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(1, len(L_i_candidate))))

            
            logp_candidate += np.log(p_a / p_b) + np.log((3 - 2 * (K_hat == 1)) * len(zero_indices) / (3 - 2 * ((K_hat + 1) == T_max)) / len(change_position_all_candidate))
            
            ratio = np.exp(min([0, logp_candidate - logp_original]))
            cxx = np.random.binomial(1, ratio)
            
            print("Cxx:",cxx,"\n")
            if cxx == 1:
                Delta_hat = Delta_hat_candidate
                Stage_hat = Stage_hat_candidate
        
        elif change_type == 0:
            
            Delta_hat_candidate = Delta_hat.copy()
            Stage_hat_candidate = Stage_hat.copy()
            
            change_position_all = np.where(np.abs(Delta_hat[1:-1] - Delta_hat[2:]) == 1)[0]+1
            change_position = np.random.choice(change_position_all, 1)

            Delta_hat_candidate[change_position + np.array([0, 1])] = Delta_hat_candidate[change_position + np.array([1, 0])]
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)
            
            index_change_candidate = np.where(Delta_hat_candidate==1)[0]
            change_position_candidate_all=np.where(np.abs(Delta_hat_candidate[1:-1] - Delta_hat_candidate[2:]) == 1)[0]+1
        
            phase = np.array([Stage_hat[change_position], Stage_hat_candidate[change_position]])
    
            ## acceptance step
            logp_original = 0
            logp_candidate = 0
        
            for i in phase:
                L_i_original = np.where(Stage_hat == i)[0]
                logp_original += np.sum(np.log(np.arange(len(L_i_original)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(len(L_i_original)) - 1 + r_shape)) - np.log(gamma_b_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(len(L_i_original))))

                L_i_candidate = np.where(Stage_hat_candidate == i)[0]
                logp_candidate += np.sum(np.log(np.arange(len(L_i_candidate)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(len(L_i_candidate)) - 1 + r_shape)) - np.log(gamma_b_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(len(L_i_candidate))))

            ratio = np.exp(min([0, logp_candidate - logp_original + np.log(len(change_position_all) / len(change_position_candidate_all))]))
            cxx = np.random.binomial(1, ratio)
            
            print("Cxx:",cxx,"\n")
            if cxx == 1:
                Delta_hat = Delta_hat_candidate
                Stage_hat = Stage_hat_candidate
        
        elif change_type == -1:
            
            Delta_hat_candidate = Delta_hat.copy()
            Stage_hat_candidate = Stage_hat.copy()
            
            one_indices = np.where(Delta_hat[1:] == 1)[0]+1
            index_to_change = np.random.choice(one_indices)

            Delta_hat_candidate[index_to_change] = 0
            Stage_hat_candidate = np.cumsum(Delta_hat_candidate)
        
            index_change_candidate = np.where(Delta_hat_candidate == 1)[0]
            change_position_all_candidate = np.where(Delta_hat_candidate[1:] == 0)[0] + 1
    
            phase_original= np.array([Stage_hat[index_to_change],Stage_hat_candidate[index_to_change]])
            phase_candidate=np.array([Stage_hat_candidate[index_to_change]])
            
            logp_original = 0
            logp_candidate = 0

            for i in phase_original:
                L_i_original = np.where(Stage_hat == i)[0]
                
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
                logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
                logp_original -= np.sum(np.log(np.arange(1, len(L_i_original))))

            for i in phase_candidate:
                L_i_candidate = np.where(Stage_hat_candidate == i)[0]
                
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_candidate)) * np.log(b_rate + np.sum(beta_hat[L_i_candidate]))
                logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
                logp_candidate -= np.sum(np.log(np.arange(1, len(L_i_candidate))))

            
            logp_candidate += np.log(p_a / p_b) + np.log((3 - 2 * (K_hat == 1)) * len(one_indices) / (3 - 2 * ((K_hat + 1) == T_max)) / len(change_position_all_candidate))

            ratio = np.exp(min([0, logp_candidate - logp_original]))
            cxx = np.random.binomial(1, ratio)
            
            print("Cxx:",cxx,"\n")
            if cxx == 1:
                Delta_hat = Delta_hat_candidate
                Stage_hat = Stage_hat_candidate
        
        ############# b and r sampling
        K_hat = np.sum(Delta_hat)
        
        b_hat = np.zeros(K_hat)
        r_hat = np.zeros(K_hat)
        for i in range(K_hat):
            L_i = np.where(Stage_hat == i)[0]
            b_hat[i] = gamma.rvs(size=1, a=(b_shape + len(L_i)), scale=1/(b_rate + np.sum(beta_hat[L_i])))
            r_hat[i] = gamma.rvs(size=1, a=(r_shape + len(L_i)), scale=1/(r_rate + np.sum(-np.log(gamma_hat[L_i]))))
    
        
        ############# beta and gamma sampling
        for t in range(T_max):
            beta_hat[t] = gamma.rvs(size=1, a=contact_hat[t] + 1, scale=1/(b_hat[Stage_hat[t]-1] + PI_obs[t]*S_obs[t]))
            gamma_hat[t] = beta.rvs(size=1, a=N_recovery_obs[t] + r_hat[Stage_hat[t]-1], b=1 + I_obs[t] - N_recovery_obs[t])

            lambda_t = beta_hat[t] * PI_obs[t]
            p_upper = 1 - poisson.cdf(0, lambda_t)
            pp = np.random.uniform(0, 1, N_infect_obs[t]) * p_upper + (1 - p_upper)
            contact_hat[t] = np.sum(poisson.ppf(pp, lambda_t))
        
        if step % 100 == 0 and step != 0:
            print("\nStep:",step)
            print("Delta_hat:",Delta_hat)
            print("b_hat:",b_hat)
            print("r_hat:",r_hat)
            print("beta_hat:",beta_hat)
            print("gamma_hat:",gamma_hat)
            print("----------------------------------------------------")
            
        Delta_all.append(Delta_hat)
        Stage_all.append(Stage_hat)
        b_all.append(b_hat)
        r_all.append(r_hat)
        beta_all.append(beta_hat)
        gamma_all.append(gamma_hat)
  
    # Creazione del DataFrame
    MCMC_chain = {
        'Delta': Delta_all[burnin::thinning],
        'Stage': Stage_all[burnin::thinning],
        'b': b_all[burnin::thinning],
        'r': r_all[burnin::thinning],
        'beta': beta_all[burnin::thinning],
        'gamma': gamma_all[burnin::thinning]
            }
    
    return MCMC_chain

In [266]:
MCMC_chain=gibbs_sampling(result)

Initialization:

Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [0.07121556]
r_hat: [1.53016829e-07]
beta_hat: [0.43768377 0.25915933 0.15355669 0.264647   0.26312357 0.20982997
 0.2544738  0.27515435 0.303436   0.31356342 0.25368889 0.32355162
 0.27784334 0.28743949 0.31229735 0.29682297 0.2910225  0.30961806
 0.29328868 0.32511483 0.27690959 0.30991405 0.30619999 0.29702406
 0.36806466 0.397862   0.46722344 0.42261832 0.41485765 0.40225606
 0.49307652 0.49357726 0.85069159 0.4268882  0.46427319 0.46267729
 0.46911989 0.41331308 0.65265645 2.04846531 0.94482293 1.26883411
 0.38449426 0.86536191 3.19297294 0.63083083 0.74484144 1.87904144
 0.4659641  1.33593902 1.96658392 1.27871138 2.35331229 0.46673893
 0.74159439 0.65505768 0.252695   0.47426907 0.4058953  0.256636
 0.42617248 0.460223   0.27040395 0.32088236 0

/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:125: RuntimeWarning: divide by zero encountered in log
  logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:132: RuntimeWarning: divide by zero encountered in log
  logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:138: RuntimeWarning: invalid value encountered in double_scalars
  ratio = np.exp(min([0, logp_candidate - logp_original]))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:23

KeyboardInterrupt: 

In [264]:
MCMC_chain=gibbs_sampling(result)

Initialization:

Delta_hat: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
b_hat: [0.00045228]
r_hat: [2.99430603e-06]
beta_hat: [2.85864773e-01 2.41904390e-01 2.91434336e-01 2.80108239e-01
 2.99256384e-01 3.41127914e-01 3.01780445e-01 4.45673724e-01
 3.75841829e-01 3.31153578e-01 4.62652503e-01 3.73536495e-01
 7.17656243e-01 4.98894640e-01 5.48307151e-01 4.31947594e-01
 3.33606280e-01 4.82565486e-01 6.54572586e-01 5.22264001e-01
 1.05415550e+00 5.32602082e-01 2.85140776e+00 8.96457581e+00
 5.86450205e+00 2.01515150e+01 8.67368455e+00 5.94966915e+00
 1.85461119e+01 1.99459176e+00 3.90996334e+00 2.43917305e+01
 4.03985953e+01 3.78883181e+00 7.23679704e+00 7.99475196e+01
 9.31203900e+01 1.42139759e+01 2.24905227e+02 1.10435818e+01
 1.94439834e+02 1.11297660e+02 2.37559336e+02 6.66664350e+01
 1.71582392e+02 1.04542694e+02 1.44870920e+

/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:125: RuntimeWarning: divide by zero encountered in log
  logp_original += np.sum(np.log(np.arange(1, len(L_i_original)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:132: RuntimeWarning: divide by zero encountered in log
  logp_candidate += np.sum(np.log(np.arange(1, len(L_i_candidate)) - 1 + r_shape)) - np.log(gamma_r_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_candidate)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_candidate])))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:138: RuntimeWarning: invalid value encountered in double_scalars
  ratio = np.exp(min([0, logp_candidate - logp_original]))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:23

Step: 1
Change_type: 1
K_hat: 2
Cxx: 0 

Step: 2
Change_type: 1
K_hat: 2
Cxx: 0 

Step: 3
Change_type: -1
K_hat: 2
Cxx: 0 

Step: 4
Change_type: 1
K_hat: 2
Cxx: 1 

Step: 5
Change_type: 1
K_hat: 3
Cxx: 1 

Step: 6
Change_type: -1
K_hat: 4
Cxx: 0 

Step: 7
Change_type: 1
K_hat: 4
Cxx: 0 

Step: 8
Change_type: 1
K_hat: 4
Cxx: 1 

Step: 9
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 10
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 11
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 12
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 13
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 14
Change_type: 0
K_hat: 5
Cxx: 1 



/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:168: RuntimeWarning: invalid value encountered in log
  logp_original += np.sum(np.log(np.arange(len(L_i_original)) - 1 + b_shape)) - np.log(gamma_b_shape) + b_shape * np.log(b_rate) - (b_shape + len(L_i_original)) * np.log(b_rate + np.sum(beta_hat[L_i_original]))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:169: RuntimeWarning: invalid value encountered in log
  logp_original += np.sum(np.log(np.arange(len(L_i_original)) - 1 + r_shape)) - np.log(gamma_b_shape) + r_shape * np.log(r_rate) - (r_shape + len(L_i_original)) * np.log(r_rate + np.sum(-np.log(gamma_hat[L_i_original])))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:170: RuntimeWarning: divide by zero encountered in log
  logp_original -= np.sum(np.log(np.arange(len(L_i_original))))
/var/folders/75/708kstf92fj1bh2chg3838140000gn/T/ipykernel_1916/2694791793.py:173: RuntimeWarning: invalid 

Step: 15
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 16
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 17
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 18
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 19
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 20
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 21
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 22
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 23
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 24
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 25
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 26
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 27
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 28
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 29
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 30
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 31
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 32
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 33
Change_type: 1
K_hat: 5
Cxx: 0 

Step: 34
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 35
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 36
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 37
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 38
Change_type: -1
K_

...

Step: 902
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 903
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 904
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 905
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 906
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 907
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 908
Change_type: -1
K_hat: 6
Cxx: 1 

Step: 909
Change_type: 0
K_hat: 5
Cxx: 1 

Step: 910
Change_type: -1
K_hat: 5
Cxx: 1 

Step: 911
Change_type: -1
K_hat: 4
Cxx: 0 

Step: 912
Change_type: -1
K_hat: 4
Cxx: 1 

Step: 913
Change_type: 1
K_hat: 3
Cxx: 1 

Step: 914
Change_type: 0
K_hat: 4
Cxx: 1 

Step: 915
Change_type: -1
K_hat: 4
Cxx: 0 

Step: 916
Change_type: 0
K_hat: 4
Cxx: 1 

Step: 917
Change_type: 1
K_hat: 4
Cxx: 0 

Step: 918
Change_type: 0
K_hat: 4
Cxx: 1 

Step: 919
Change_type: -1
K_hat: 4
Cxx: 0 

Step: 920
Change_type: -1
K_hat: 4
Cxx: 0 

Step: 921
Change_type: 0
K_hat: 4
Cxx: 1 

Step: 922
Change_type: 1
K_hat: 4
Cxx: 0 

Step: 923
Change_type: -1
K_hat: 4
Cxx: 0 

Step: 924
Change_type: -1
K_hat: 4
Cxx: 1 

S

Step: 1028
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1029
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1030
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1031
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1032
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1033
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1034
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1035
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1036
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1037
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1038
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1039
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1040
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1041
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1042
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1043
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1044
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1045
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1046
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1047
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1048
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1049
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1050
Change_type

Step: 1152
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1153
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1154
Change_type: 1
K_hat: 6
Cxx: 0 

Step: 1155
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1156
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1157
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1158
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1159
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1160
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1161
Change_type: -1
K_hat: 6
Cxx: 0 

Step: 1162
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1163
Change_type: 0
K_hat: 6
Cxx: 1 

Step: 1164
Change_type: -1
K_hat: 6
Cxx: 1 

Step: 1165
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 1166
Change_type: -1
K_hat: 5
Cxx: 1 

Step: 1167
Change_type: 0
K_hat: 4
Cxx: 1 

Step: 1168
Change_type: -1
K_hat: 4
Cxx: 1 

Step: 1169
Change_type: 1
K_hat: 3
Cxx: 0 

Step: 1170
Change_type: 1
K_hat: 3
Cxx: 0 

Step: 1171
Change_type: 1
K_hat: 3
Cxx: 1 

Step: 1172
Change_type: 1
K_hat: 4
Cxx: 1 

Step: 1173
Change_type: -1
K_hat: 5
Cxx: 0 

Step: 1174
Change_type:

Step: 1276
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1277
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1278
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1279
Change_type: 0
K_hat: 7
Cxx: 1 

Step: 1280
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1281
Change_type: 0
K_hat: 7
Cxx: 1 

Step: 1282
Change_type: 0
K_hat: 7
Cxx: 1 

Step: 1283
Change_type: 0
K_hat: 7
Cxx: 1 

Step: 1284
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1285
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1286
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1287
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1288
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1289
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1290
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1291
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1292
Change_type: 0
K_hat: 7
Cxx: 1 

Step: 1293
Change_type: 0
K_hat: 7
Cxx: 1 

Step: 1294
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1295
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1296
Change_type: -1
K_hat: 7
Cxx: 0 

Step: 1297
Change_type: 1
K_hat: 7
Cxx: 0 

Step: 1298
Change_type: 0

KeyboardInterrupt: 